In [1]:
import glob
import pandas as pd
import numpy as np
from sklearn.utils import shuffle

import matplotlib.pylab as plt
from tqdm import tqdm
import tensorflow as tf
from tensorflow.keras import *

In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

2 Physical GPUs, 2 Logical GPUs


In [3]:
strategy = tf.distribute.MirroredStrategy()

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')


In [4]:
LEARNING_RATE = 0.002
EPSILON = 1e-08
BEST_PATH = './models/ResUNet.h5'

In [5]:
train_files = glob.glob('./data/train/*.npy')
train_files = shuffle(train_files, random_state=3101)
len(train_files)

62735

In [6]:
def trainGenerator():
    for file in train_files:
        dataset = np.load(file)
        target= dataset[:,:,-1].reshape(120,120,1)
        remove_minus = np.where(target < 0, 0, target)
        feature = dataset[:,:,:4]

        yield (feature, remove_minus)

In [7]:
# NUM_TRAIN = int(len(train_files)*.7)

# full_dataset = tf.data.Dataset.from_generator(trainGenerator, (tf.float32, tf.float32), (tf.TensorShape([120,120,4]),tf.TensorShape([120,120,1])))

# train_dataset = full_dataset.take(NUM_TRAIN)
# train_dataset = train_dataset.batch(64).prefetch(1).repeat()
# val_dataset = full_dataset.skip(NUM_TRAIN)
# val_dataset = val_dataset.batch(64).prefetch(1).repeat()

In [8]:
NUM_TRAIN = int(len(train_files)*.7)

full_dataset = tf.data.Dataset.from_generator(trainGenerator, (tf.float32, tf.float32), (tf.TensorShape([120,120,4]),tf.TensorShape([120,120,1])))

train_dataset = full_dataset.take(NUM_TRAIN)
train_dataset = train_dataset.batch(64).prefetch(1)
val_dataset = full_dataset.skip(NUM_TRAIN)
val_dataset = val_dataset.batch(64).prefetch(1)

In [9]:
class InitConvBlock(layers.Layer):
    def __init__(self, filters, kernel_size):
        super(InitConvBlock, self).__init__()
        self.f = filters
        self.k = kernel_size
        
        self.downsampling = layers.Conv2D(self.f, (1, 1), kernel_initializer='he_normal', padding='same')
        self.downbatch = layers.BatchNormalization()
        
        self.conv1 = layers.Conv2D(self.f, self.k, kernel_initializer='he_normal', padding='same')
        self.batch1 = layers.BatchNormalization()
        self.activation1 = layers.Activation(tf.nn.leaky_relu)
        self.conv2 = layers.Conv2D(self.f, self.k, kernel_initializer='he_normal', padding='same')
        
    def call(self, inp, TRAINING):

        shortcut = self.downbatch(self.downsampling(inp), training=TRAINING)

        inp = self.conv2(self.activation1(self.batch1(self.conv1(inp), training=TRAINING)))
        inp = layers.add([shortcut, inp])
        
        return inp

In [10]:
class ConvBlock(layers.Layer):
    def __init__(self, filters, kernel_size):
        super(ConvBlock, self).__init__()
        self.f = filters
        self.k = kernel_size
        
        self.downsampling = layers.Conv2D(self.f, (1, 1), kernel_initializer='he_normal', padding='same')
        self.downbatch = layers.BatchNormalization()
        
        self.batch1 = layers.BatchNormalization()
        self.activation1 = layers.Activation(tf.nn.leaky_relu)
        self.conv1 = layers.Conv2D(self.f/4, (1, 1), kernel_initializer='he_normal', padding='same')
        
        self.batch2 = layers.BatchNormalization()
        self.activation2 = layers.Activation(tf.nn.leaky_relu)
        self.conv2 = layers.Conv2D(self.f/4, self.k, kernel_initializer='he_normal', padding='same')
        
        self.outconv = layers.Conv2D(self.f, (1, 1), kernel_initializer='he_normal', padding='same')
        self.outbatch = layers.BatchNormalization()
        self.outact = layers.Activation(tf.nn.leaky_relu)
        
    def call(self, inp, TRAINING):
        
        shortcut = self.downbatch(self.downsampling(inp), training=TRAINING)
        
        inp = self.conv1(self.activation1(self.batch1(inp, training=TRAINING)))
        inp = self.conv2(self.activation2(self.batch2(inp, training=TRAINING)))
        inp = self.outbatch(self.outconv(inp), training=TRAINING)
        inp = self.outact(layers.add([shortcut, inp]))
                
        return inp

In [11]:
class DeconvBlock(layers.Layer):
    def __init__(self, filters, kernel_size, strides):
        super(DeconvBlock, self).__init__()
        self.f = filters
        self.k = kernel_size
        self.s = strides
        
        self.deconv1 = layers.Conv2DTranspose(self.f, self.k, kernel_initializer='he_normal', strides=self.s, padding='same')
        self.activation1 = layers.Activation(tf.nn.leaky_relu)
        
    def call(self, inp):
        
        inp = self.activation1(self.deconv1(inp))
        
        return inp

In [12]:
class UNet(Model):
    def __init__(self):
        super(UNet, self).__init__()
        self.n = [64, 128, 256, 512, 1024] # number of nodes
        self.k = (3, 3) # kernal size
        self.s = (2, 2) # stride (= pooling size)
        
        
        self.conv_block1 = InitConvBlock(self.n[0], self.k)
        self.pool1 = layers.Conv2D(self.n[1], self.s, self.s)
        
        self.conv_block2 = ConvBlock(self.n[1], self.k)
        self.pool2 = layers.Conv2D(self.n[2], self.s, self.s)
        
        self.conv_block3 = ConvBlock(self.n[2], self.k)
        self.pool3 = layers.Conv2D(self.n[3], self.s, self.s)
        
        self.conv_block4 = ConvBlock(self.n[3], self.k)
        self.pool4 = layers.Conv2D(self.n[4], self.s, self.s)
        
        self.conv_bottom = ConvBlock(self.n[4], self.k)
        
        self.deconv_block1 = layers.UpSampling2D()
        self.conv_block_r1 = ConvBlock(self.n[3], self.k)
        
        self.deconv_block2 = layers.UpSampling2D()
        self.conv_block_r2 = ConvBlock(self.n[2], self.k)
        
        self.deconv_block3 = layers.UpSampling2D()
        self.conv_block_r3 = ConvBlock(self.n[1], self.k)
        
        self.deconv_block4 = layers.UpSampling2D()
        self.conv_block_r4 = ConvBlock(self.n[0], self.k)
        
        self.padding = layers.ZeroPadding2D(((1, 0), (0, 1)))
        self.output_conv = layers.Conv2D(1, (1, 1), activation='sigmoid')
        
    def call(self, inp, TRAINING=True):
        inp = inp/255
        
        conv1 = self.conv_block1(inp, TRAINING)
        pooled1 = self.pool1(conv1)
        
        conv2 = self.conv_block2(pooled1, TRAINING)
        pooled2 = self.pool2(conv2)
        
        conv3 = self.conv_block3(pooled2, TRAINING)
        pooled3 = self.pool3(conv3)
        
        conv4 = self.conv_block4(pooled3, TRAINING)
        pooled4 = self.pool4(conv4)
        
        bottom = self.conv_bottom(pooled4, TRAINING)
        
        deconv1 = self.padding(self.deconv_block1(bottom))
        deconv1 = layers.concatenate([deconv1, conv4])
        deconv1 = self.conv_block_r1(deconv1, TRAINING)
        
        deconv2 = self.deconv_block2(deconv1)
        deconv2 = layers.concatenate([deconv2, conv3])
        deconv2 = self.conv_block_r2(deconv2, TRAINING)
        
        deconv3 = self.deconv_block3(deconv2)
        deconv3 = layers.concatenate([deconv3, conv2])
        deconv3 = self.conv_block_r3(deconv3, TRAINING)
        
        deconv4 = self.deconv_block4(deconv3)
        deconv4 = layers.concatenate([deconv4, conv1])
        deconv4 = self.conv_block_r4(deconv4, TRAINING)
        
        return self.output_conv(deconv4)*255

In [13]:
callbacks = [
    tf.keras.callbacks.ModelCheckpoint(
        filepath=BEST_PATH,
        # Path where to save the model
        # The two parameters below mean that we will overwrite
        # the current checkpoint if and only if
        # the `val_loss` score has improved.
        save_best_only=True,
        monitor='val_loss',
        verbose=1),
    tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0.0001,  patience=20)
]

In [14]:
# model.compile(loss='mae', optimizer=opt)
# model.fit(train_dataset, epochs = 200,
#           callbacks = callbacks, validation_data=val_dataset,
#           steps_per_epoch = 100, validation_steps=30,
#           verbose=1)

In [15]:
with strategy.scope():
    opt = tf.optimizers.Adam(learning_rate=LEARNING_RATE/2, epsilon=EPSILON)
    model = UNet()
    model.compile(loss='mae', optimizer=opt)
    model.fit(train_dataset, epochs = 200,
              callbacks = callbacks, validation_data=val_dataset,
              verbose=1)

Epoch 1/200
Instructions for updating:
Use `tf.data.Iterator.get_next_as_optional()` instead.
INFO:tensorflow:batch_all_reduce: 164 all-reduces with algorithm = nccl, num_packs = 1
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:batch_all_reduce: 164 all-reduces with algorithm = nccl, num_packs = 1
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
    687/Unknown - 118s 172ms/step - loss: 4.2623INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/d

KeyboardInterrupt: 

In [ ]:
with strategy.scope():
    model.load_weights(BEST_PATH)

In [ ]:
test_path = './data/test'
test_files = sorted(glob.glob(test_path + '/*.npy'))

X_test = []

for file in tqdm(test_files, desc = 'test'):
    data = np.load(file)
    X_test.append(data)

X_test = np.array(X_test)

In [ ]:
with strategy.scope():
    pred = model.predict(X_test.astype('float32'))

In [ ]:
submission = pd.read_csv('./data/sample_submission.csv')

In [ ]:
submission.iloc[:,1:] = pred.reshape(-1, 14400).astype(int)
submission.to_csv('./results/Dacon_UNet_1.csv', index = False)